In [38]:
!pip -q install fastai2 optuna swifter toolz

In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
tf.config.list_physical_devices('GPU')
tf.config.experimental.enable_op_determinism()
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
SEED = 7
import matplotlib.pyplot as plt

In [40]:
# Imprimir los dispositivos físicos disponibles (GPU)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [41]:
# Cargar el conjunto de datos
datos = pd.read_csv("/content/sample_data/Temp_Asu20092021.csv")

In [42]:
# Convertir la columna 'Fecha' al tipo datetime
datos['Fecha'] = pd.to_datetime(datos['Fecha'])

In [43]:
datos

,Unnamed: 0,Fecha,Temperatura,Humedad
0,0,2009-01-01 00:00:00,26.749483,69.609451
1,1,2009-01-01 03:00:00,25.956114,78.225166
2,2,2009-01-01 06:00:00,23.707357,93.551756
3,3,2009-01-01 09:00:00,23.726823,93.394817
4,4,2009-01-01 12:00:00,29.606379,72.155839
...,...,...,...,...
37979,37979,2021-12-31 09:00:00,21.719499,85.092387
37980,37980,2021-12-31 12:00:00,29.548918,53.780205
37981,37981,2021-12-31 15:00:00,40.026148,27.392646
37982,37982,2021-12-31 18:00:00,41.244712,19.955186


In [44]:
# Hacer que la 'Fecha' sea el índice
# COMPLETAR
datos.set_index("Fecha",inplace=True)

In [45]:
datos=datos.drop(datos.columns[0], axis=1)

In [46]:
datos

,Temperatura,Humedad
Fecha,,
2009-01-01 00:00:00,26.749483,69.609451
2009-01-01 03:00:00,25.956114,78.225166
2009-01-01 06:00:00,23.707357,93.551756
2009-01-01 09:00:00,23.726823,93.394817
2009-01-01 12:00:00,29.606379,72.155839
...,...,...
2021-12-31 09:00:00,21.719499,85.092387
2021-12-31 12:00:00,29.548918,53.780205
2021-12-31 15:00:00,40.026148,27.392646


In [47]:
datos = datos[datos.index.year >= 2019]

In [48]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

<ipython-input-48-e58a81f22bbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.dropna(inplace=True)


In [49]:
datos

,Temperatura,Humedad
Fecha,,
2019-01-01 00:00:00,28.260936,66.311373
2019-01-01 03:00:00,27.051418,68.882454
2019-01-01 06:00:00,26.234018,72.660367
2019-01-01 09:00:00,25.644533,74.647767
2019-01-01 12:00:00,31.437436,55.215384
...,...,...
2021-12-31 09:00:00,21.719499,85.092387
2021-12-31 12:00:00,29.548918,53.780205
2021-12-31 15:00:00,40.026148,27.392646


In [50]:
# Para cambiar de muestreo de 3 horas a 1 hora interpolando valores
datos_interpolados = datos.resample("H").interpolate(method='spline', order=3)

In [51]:
datos_interpolados

,Temperatura,Humedad
Fecha,,
2019-01-01 00:00:00,28.260936,66.311373
2019-01-01 01:00:00,28.223146,66.523607
2019-01-01 02:00:00,27.908703,67.385359
2019-01-01 03:00:00,27.051418,68.882454
2019-01-01 04:00:00,26.729742,70.296842
...,...,...
2021-12-31 17:00:00,41.562780,19.987211
2021-12-31 18:00:00,41.244712,19.955186
2021-12-31 19:00:00,41.059331,21.839127


In [52]:
datos = datos_interpolados
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

In [53]:
datos

,Temperatura,Humedad
Fecha,,
2019-01-01 00:00:00,28.260936,66.311373
2019-01-01 01:00:00,28.223146,66.523607
2019-01-01 02:00:00,27.908703,67.385359
2019-01-01 03:00:00,27.051418,68.882454
2019-01-01 04:00:00,26.729742,70.296842
...,...,...
2021-12-31 17:00:00,41.562780,19.987211
2021-12-31 18:00:00,41.244712,19.955186
2021-12-31 19:00:00,41.059331,21.839127


In [54]:
# CALCULAR EL MÁXIMO DE CADA DÍA
max_temperaturas_diarias = datos.resample("D").max()
max_temperaturas_diarias = max_temperaturas_diarias.drop(max_temperaturas_diarias.columns[1],axis=1)
max_temperaturas_diarias

,Temperatura
Fecha,
2019-01-01,37.639718
2019-01-02,38.286905
2019-01-03,38.163652
2019-01-04,37.351168
2019-01-05,34.843885
...,...
2021-12-27,40.380080
2021-12-28,41.448426
2021-12-29,42.678048


In [55]:
# Aplicar corte de cuartil a las temperaturas máximas diarias
cuartiles = max_temperaturas_diarias.quantile([0.25, 0.5, 0.75], axis=0)

In [56]:
cuartiles

,Temperatura
0.25,27.066809
0.50,31.893574
0.75,36.407532


In [57]:
# Definir categorías basadas en los cuartiles
umbral_frio = cuartiles["Temperatura"][0.25]
umbral_bueno = cuartiles["Temperatura"][0.5]
umbral_caliente = cuartiles["Temperatura"][0.75]
print(cuartiles)

      Temperatura
0.25    27.066809
0.50    31.893574
0.75    36.407532


In [58]:
datos['Max_Temperatura_Dia'] = datos.groupby(datos.index.date)['Temperatura'].transform('max')

In [59]:
datos['Categoria_Temperatura'] = pd.cut(datos['Max_Temperatura_Dia'], bins=[-float('inf'), umbral_frio, umbral_bueno, umbral_caliente, float('inf')],
                                      labels=['Frío', 'Bueno', 'Caliente', 'Muy Caliente'])
# Asignar cada observación de temperatura a su categoría correspondiente

In [60]:
datos

,Temperatura,Humedad,Max_Temperatura_Dia,Categoria_Temperatura
Fecha,,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718,Muy Caliente
2019-01-01 01:00:00,28.223146,66.523607,37.639718,Muy Caliente
2019-01-01 02:00:00,27.908703,67.385359,37.639718,Muy Caliente
2019-01-01 03:00:00,27.051418,68.882454,37.639718,Muy Caliente
2019-01-01 04:00:00,26.729742,70.296842,37.639718,Muy Caliente
...,...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780,Muy Caliente
2021-12-31 18:00:00,41.244712,19.955186,41.562780,Muy Caliente
2021-12-31 19:00:00,41.059331,21.839127,41.562780,Muy Caliente


In [61]:
# Desplazar para obtener la temperatura del siguiente día
datos['Temperatura_Siguiente_Dia'] = datos['Categoria_Temperatura'].shift(-24)

In [62]:
# USAR ONE HOT ENCODING
datos_codificados = pd.get_dummies(datos, columns=None)

In [63]:
datos_codificados

,Temperatura,Humedad,Max_Temperatura_Dia,Categoria_Temperatura_Frío,Categoria_Temperatura_Bueno,Categoria_Temperatura_Caliente,Categoria_Temperatura_Muy Caliente,Temperatura_Siguiente_Dia_Frío,Temperatura_Siguiente_Dia_Bueno,Temperatura_Siguiente_Dia_Caliente,Temperatura_Siguiente_Dia_Muy Caliente
Fecha,,,,,,,,,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718,False,False,False,True,False,False,False,True
2019-01-01 01:00:00,28.223146,66.523607,37.639718,False,False,False,True,False,False,False,True
2019-01-01 02:00:00,27.908703,67.385359,37.639718,False,False,False,True,False,False,False,True
2019-01-01 03:00:00,27.051418,68.882454,37.639718,False,False,False,True,False,False,False,True
2019-01-01 04:00:00,26.729742,70.296842,37.639718,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780,False,False,False,True,False,False,False,False
2021-12-31 18:00:00,41.244712,19.955186,41.562780,False,False,False,True,False,False,False,False
2021-12-31 19:00:00,41.059331,21.839127,41.562780,False,False,False,True,False,False,False,False


In [64]:
# Definir características y variable objetivo
X = datos_codificados.iloc[:,1:7]# Probar diferentes combinaciones de features
y = datos_codificados.iloc[:,6:10] #para simplificar el problema tambien se podria hacer un modelo que prediga solo dias frios y muy calientes


In [65]:
# Normalizar características numéricas
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)

In [66]:
# Dividir datos en conjuntos de entrenamiento y validación
indices_entrenamiento = (datos.index.year <= 2019)
indices_validacion = ((datos.index.year >= 2020) & (datos.index.year <= 2020))

In [67]:
# Dividir datos en conjuntos de entrenamiento y validación
X_entrenamiento, X_validacion = X_escalado[indices_entrenamiento], X_escalado[indices_validacion]

In [68]:
X_entrenamiento

array([[ 0.32631727,  0.97821563, -0.57737954, -0.57737954, -0.57737954,
         1.73231426],
       [ 0.33614881,  0.97821563, -0.57737954, -0.57737954, -0.57737954,
         1.73231426],
       [ 0.37606864,  0.97821563, -0.57737954, -0.57737954, -0.57737954,
         1.73231426],
       ...,
       [ 1.63594114,  1.33146938, -0.57737954, -0.57737954, -0.57737954,
         1.73231426],
       [ 1.61285821,  1.33146938, -0.57737954, -0.57737954, -0.57737954,
         1.73231426],
       [ 1.26608181,  1.33146938, -0.57737954, -0.57737954, -0.57737954,
         1.73231426]])

In [69]:
y_entrenamiento, y_validacion =y[indices_entrenamiento],y[indices_validacion]

In [71]:
# Definir función objetivo para Optuna
def objetivo(trial):
    # Definir hiperparámetros a ajustar
    num_capas = 1
    tf.keras.utils.set_random_seed(SEED)

    num_unidades = trial.suggest_categorical('num_unidades', [32, 64,128]) # modificar
    tasa_aprendizaje =trial.suggest_categorical('tasa_aprendizaje', [1e-1, 1e-2,1e-3,1e-4]) # modificar
    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X.shape[1],)))
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu')) # se podria agregar una capa de dropout
    modelo.add(Dense(4, activation='softmax'))  # 4 categorías: Frío, Bueno, Caliente, Muy Caliente, aqui se pueden probar otras opciones

    # Compilar el modelo
    modelo.compile(optimizer=Adam(learning_rate=tasa_aprendizaje),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # Entrenar el modelo  # el batch size se puede variar tambien depediendo de la gpu que se tenga, el numero de épocas debe ser de, al menos 50 o 100
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=50, batch_size=256, verbose=0, shuffle=False)


    # Evaluar el modelo en el conjunto de validación
    _, val_acc = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_acc

In [ ]:
# Realizar la optimización de hiperparámetros usando Optuna
estudio = optuna.create_study(direction='maximize') # cambiar si se modificar la metrica de la funcion objetivo, se puede hacer un problema multiobjetivo tambien
estudio.optimize(objetivo, n_trials=50) # probar mas trials una vez que se tenga el script funcionando, minimo 50, evitar variar muchos parametros al mismo tiempo

# Obtener los mejores hiperparámetros
mejor_num_capas = 1
mejor_num_unidades = estudio.best_params['num_unidades']
mejor_tasa_aprendizaje = estudio.best_params['tasa_aprendizaje']

print("Mejores Hiperparámetros:")
print("Número de Capas:", mejor_num_capas)
print("Número de Unidades:", mejor_num_unidades)
print("Tasa de Aprendizaje:", mejor_tasa_aprendizaje)

# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(4, activation='softmax'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=mejor_tasa_aprendizaje),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Dividir datos en conjuntos de entrenamiento y prueba
indices_entrenamiento_final = (datos.index.year < 2020)
indices_prueba_final = (datos.index.year >= 2021)

# Dividir datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba = X_escalado[indices_entrenamiento_final], X_escalado[indices_prueba_final]
y_entrenamiento, y_prueba = y[indices_entrenamiento_final],y[indices_prueba_final]



# Entrenar modelo final
modelo_final.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_prueba, y_prueba ), epochs=50, batch_size=256, verbose=1,shuffle=False)

#Evaluar en conjunto de prueba
_, val_acc =modelo_final.evaluate(X_prueba, y_prueba, verbose=0)
print(val_acc)

# Ejemplo de como calcular las otras metricas que podrias utilizarse como funcion objetivo para la optimizacion de hiperparametros en lugar del acc
y_pred = modelo_final.predict(X_prueba)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir las predicciones en clases

precision = precision_score(np.argmax(y_prueba, axis=1), y_pred_classes, average='weighted')
recall = recall_score(np.argmax(y_prueba, axis=1), y_pred_classes, average='weighted')
f1 = f1_score(np.argmax(y_prueba, axis=1), y_pred_classes, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

[I 2024-05-02 21:44:51,396] A new study created in memory with name: no-name-82af4963-b233-4082-9a8f-4537dfedf616
[I 2024-05-02 21:45:14,004] Trial 0 finished with value: 0.6177140474319458 and parameters: {'num_unidades': 128, 'tasa_aprendizaje': 0.01}. Best is trial 0 with value: 0.6177140474319458.
[I 2024-05-02 21:45:25,532] Trial 1 finished with value: 0.6664389967918396 and parameters: {'num_unidades': 32, 'tasa_aprendizaje': 0.001}. Best is trial 1 with value: 0.6664389967918396.
[I 2024-05-02 21:45:35,793] Trial 2 finished with value: 0.5934653878211975 and parameters: {'num_unidades': 64, 'tasa_aprendizaje': 0.1}. Best is trial 1 with value: 0.6664389967918396.
[I 2024-05-02 21:45:47,634] Trial 3 finished with value: 0.618852436542511 and parameters: {'num_unidades': 32, 'tasa_aprendizaje': 0.01}. Best is trial 1 with value: 0.6664389967918396.
[I 2024-05-02 21:45:59,171] Trial 4 finished with value: 0.618852436542511 and parameters: {'num_unidades': 32, 'tasa_aprendizaje': 0.